# Project 

## Todo list 
- [ ] Explore the dataset
- [ ] More than one prediction model
- [ ] Feature engineering 
- [ ] Model interpretation
- [ ] Create short notebook


## Data exploration
- Search domain knowledge 
- Check if the data is intuitive
- Explore individual features
- Explore pairs and groups of features
- Clean up features

In [ ]:
## TODO CODE HERE

## Feature engineering

In [ ]:
## TODO CODE HERE

## Prediction model 1 

In [ ]:
## TODO CODE HERE

## Prediction model 2

In [ ]:
## TODO CODE HERE

## Model interpretation

In [ ]:
## TODO CODE HERE